In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Bidirectional, LSTM, Reshape, RepeatVector, TimeDistributed
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

Using TensorFlow backend.


In [2]:
def load_data():
        x_train = np.load(r'C:\Users\Vee\Desktop\python\GAN\Music GAN\songs.npy',allow_pickle=True)
        x_train = x_train.reshape(len(x_train),4,4)
        return x_train

In [3]:
class LSTMGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 4
        self.img_cols = 4
        self.img_shape = (self.img_rows, self.img_cols)
        self.latent_dim = 16

        optimizer = Adam(0.0001, 0.4)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(4,4))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Bidirectional(LSTM(160, return_sequences=True), input_shape=(4, 4)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Bidirectional(LSTM(160, return_sequences=True)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Bidirectional(LSTM(160)))
        model.add(LeakyReLU(alpha=0.2))
        #specifying output to have 40 timesteps
        model.add(RepeatVector(16))
        #specifying 1 feature as the output
        model.add(Bidirectional(LSTM(160, return_sequences=True, dropout = 0.2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Bidirectional(LSTM(160, return_sequences=True, dropout = 0.2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Bidirectional(LSTM(160, return_sequences=True, dropout = 0.2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.3))   
        model.add(TimeDistributed(Dense(160)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(TimeDistributed(Dense(160)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(TimeDistributed(Dense(1)))
        model.add(LeakyReLU(alpha=0.2))
        model.summary()

        noise = Input(shape=(4,4))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Bidirectional(LSTM(160, return_sequences=True), input_shape=(16, 1)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Bidirectional(LSTM(160)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(RepeatVector(1))
        model.add(TimeDistributed(Dense(160)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(TimeDistributed(Dense(160)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(TimeDistributed(Dense(1)))
        model.summary()

        img = Input(shape=(16,1))
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = load_data()

        # Rescale -1 to 1
        X_train = X_train / 128

        # Adversarial ground truths
        valid = np.ones((batch_size,1,1))
        fake = np.zeros((batch_size,1,1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            imgs = np.array(imgs)
            imgs = imgs.reshape(len(imgs),16,1)

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size,4,4))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.generator.save("LSTM_generator1.h5")

In [4]:
lstmgan = LSTMGAN()
lstmgan.train(epochs=1001, batch_size=40, save_interval=100)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 16, 320)           207360    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 320)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 320)               615680    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 320)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 320)               0         
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 1, 320)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 160)           

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 7.350955, acc.: 50.00%] [G loss: 6.289051]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 3.129720, acc.: 50.00%] [G loss: 5.826455]
2 [D loss: 2.928718, acc.: 50.00%] [G loss: 5.594081]
3 [D loss: 2.825413, acc.: 50.00%] [G loss: 5.406894]
4 [D loss: 2.731702, acc.: 50.00%] [G loss: 5.294282]
5 [D loss: 2.655751, acc.: 50.00%] [G loss: 5.144709]
6 [D loss: 2.597709, acc.: 50.00%] [G loss: 5.103446]
7 [D loss: 2.553933, acc.: 50.00%] [G loss: 4.971944]
8 [D loss: 2.477453, acc.: 50.00%] [G loss: 4.849258]
9 [D loss: 2.405779, acc.: 50.00%] [G loss: 4.753980]
10 [D loss: 2.367964, acc.: 50.00%] [G loss: 4.682136]
11 [D loss: 2.347654, acc.: 50.00%] [G loss: 4.627163]
12 [D loss: 2.309519, acc.: 50.00%] [G loss: 4.481177]
13 [D loss: 2.272539, acc.: 50.00%] [G loss: 4.441154]
14 [D loss: 2.249113, acc.: 50.00%] [G loss: 4.394472]
15 [D loss: 2.180459, acc.: 50.00%] [G loss: 4.335694]
16 [D loss: 2.141166, acc.: 50.00%] [G loss: 4.205614]
17 [D loss: 2.120231, acc.: 50.00%] [G loss: 4.174664]
18 [D loss: 2.084530, acc.: 50.00%] [G loss: 4.055471]
19 [D loss: 2.06215

...

297 [D loss: 0.737064, acc.: 37.50%] [G loss: 0.802394]
298 [D loss: 0.766250, acc.: 35.00%] [G loss: 0.788006]
299 [D loss: 0.728366, acc.: 46.25%] [G loss: 0.828989]
300 [D loss: 0.727783, acc.: 52.50%] [G loss: 0.791333]
301 [D loss: 0.765643, acc.: 43.75%] [G loss: 0.758970]
302 [D loss: 0.718334, acc.: 42.50%] [G loss: 0.772771]
303 [D loss: 0.739874, acc.: 45.00%] [G loss: 0.804739]
304 [D loss: 0.745814, acc.: 47.50%] [G loss: 0.812010]
305 [D loss: 0.733071, acc.: 46.25%] [G loss: 0.780859]
306 [D loss: 0.686564, acc.: 60.00%] [G loss: 0.819390]
307 [D loss: 0.706257, acc.: 48.75%] [G loss: 0.764173]
308 [D loss: 0.774022, acc.: 38.75%] [G loss: 0.784164]
309 [D loss: 0.736348, acc.: 48.75%] [G loss: 0.788109]
310 [D loss: 0.695478, acc.: 53.75%] [G loss: 0.781896]
311 [D loss: 0.723680, acc.: 46.25%] [G loss: 0.820045]
312 [D loss: 0.738310, acc.: 46.25%] [G loss: 0.748634]
313 [D loss: 0.737150, acc.: 48.75%] [G loss: 0.791084]
314 [D loss: 0.724838, acc.: 48.75%] [G loss: 0.

444 [D loss: 0.760544, acc.: 37.50%] [G loss: 0.726516]
445 [D loss: 0.730081, acc.: 45.00%] [G loss: 0.769558]
446 [D loss: 0.745187, acc.: 41.25%] [G loss: 0.852853]
447 [D loss: 0.695938, acc.: 51.25%] [G loss: 0.803717]
448 [D loss: 0.718131, acc.: 50.00%] [G loss: 0.775113]
449 [D loss: 0.714238, acc.: 42.50%] [G loss: 0.739248]
450 [D loss: 0.675510, acc.: 53.75%] [G loss: 0.739588]
451 [D loss: 0.749993, acc.: 42.50%] [G loss: 0.780425]
452 [D loss: 0.711980, acc.: 51.25%] [G loss: 0.777367]
453 [D loss: 0.742979, acc.: 42.50%] [G loss: 0.795889]
454 [D loss: 0.714476, acc.: 42.50%] [G loss: 0.749510]
455 [D loss: 0.718182, acc.: 47.50%] [G loss: 0.735536]
456 [D loss: 0.686900, acc.: 56.25%] [G loss: 0.789914]
457 [D loss: 0.713226, acc.: 51.25%] [G loss: 0.763884]
458 [D loss: 0.745151, acc.: 37.50%] [G loss: 0.745402]
459 [D loss: 0.754401, acc.: 37.50%] [G loss: 0.774110]
460 [D loss: 0.705907, acc.: 51.25%] [G loss: 0.819781]
461 [D loss: 0.764202, acc.: 41.25%] [G loss: 0.

591 [D loss: 0.756504, acc.: 40.00%] [G loss: 0.808268]
592 [D loss: 0.719278, acc.: 40.00%] [G loss: 0.845229]
593 [D loss: 0.749280, acc.: 43.75%] [G loss: 0.811628]
594 [D loss: 0.731402, acc.: 46.25%] [G loss: 0.808179]
595 [D loss: 0.728243, acc.: 42.50%] [G loss: 0.783468]
596 [D loss: 0.749127, acc.: 40.00%] [G loss: 0.779215]
597 [D loss: 0.723506, acc.: 50.00%] [G loss: 0.816638]
598 [D loss: 0.706061, acc.: 48.75%] [G loss: 0.810441]
599 [D loss: 0.734855, acc.: 41.25%] [G loss: 0.789597]
600 [D loss: 0.727691, acc.: 47.50%] [G loss: 0.814744]
601 [D loss: 0.712505, acc.: 45.00%] [G loss: 0.789998]
602 [D loss: 0.744479, acc.: 43.75%] [G loss: 0.757590]
603 [D loss: 0.732906, acc.: 46.25%] [G loss: 0.757306]
604 [D loss: 0.734518, acc.: 38.75%] [G loss: 0.781717]
605 [D loss: 0.697585, acc.: 47.50%] [G loss: 0.797658]
606 [D loss: 0.722084, acc.: 51.25%] [G loss: 0.764922]
607 [D loss: 0.767992, acc.: 40.00%] [G loss: 0.793518]
608 [D loss: 0.689077, acc.: 55.00%] [G loss: 0.

738 [D loss: 0.738346, acc.: 43.75%] [G loss: 0.790543]
739 [D loss: 0.694362, acc.: 50.00%] [G loss: 0.725959]
740 [D loss: 0.766979, acc.: 35.00%] [G loss: 0.839900]
741 [D loss: 0.721536, acc.: 47.50%] [G loss: 0.779534]
742 [D loss: 0.733537, acc.: 32.50%] [G loss: 0.789264]
743 [D loss: 0.722675, acc.: 51.25%] [G loss: 0.770489]
744 [D loss: 0.723121, acc.: 42.50%] [G loss: 0.747773]
745 [D loss: 0.701876, acc.: 47.50%] [G loss: 0.763227]
746 [D loss: 0.737218, acc.: 42.50%] [G loss: 0.763448]
747 [D loss: 0.723898, acc.: 48.75%] [G loss: 0.760991]
748 [D loss: 0.727372, acc.: 45.00%] [G loss: 0.785934]
749 [D loss: 0.752865, acc.: 38.75%] [G loss: 0.747209]
750 [D loss: 0.747640, acc.: 40.00%] [G loss: 0.758812]
751 [D loss: 0.714162, acc.: 51.25%] [G loss: 0.825136]
752 [D loss: 0.763275, acc.: 31.25%] [G loss: 0.791058]
753 [D loss: 0.723528, acc.: 50.00%] [G loss: 0.800050]
754 [D loss: 0.707989, acc.: 48.75%] [G loss: 0.771407]
755 [D loss: 0.762448, acc.: 40.00%] [G loss: 0.

885 [D loss: 0.744457, acc.: 43.75%] [G loss: 0.788950]
886 [D loss: 0.733330, acc.: 43.75%] [G loss: 0.791884]
887 [D loss: 0.710806, acc.: 52.50%] [G loss: 0.716539]
888 [D loss: 0.757738, acc.: 35.00%] [G loss: 0.763314]
889 [D loss: 0.756618, acc.: 37.50%] [G loss: 0.798935]
890 [D loss: 0.744588, acc.: 41.25%] [G loss: 0.772416]
891 [D loss: 0.709501, acc.: 47.50%] [G loss: 0.791540]
892 [D loss: 0.740668, acc.: 42.50%] [G loss: 0.835621]
893 [D loss: 0.712259, acc.: 51.25%] [G loss: 0.787794]
894 [D loss: 0.728090, acc.: 48.75%] [G loss: 0.813519]
895 [D loss: 0.727206, acc.: 43.75%] [G loss: 0.754462]
896 [D loss: 0.739767, acc.: 33.75%] [G loss: 0.760702]
897 [D loss: 0.717226, acc.: 48.75%] [G loss: 0.764627]
898 [D loss: 0.737438, acc.: 43.75%] [G loss: 0.805173]
899 [D loss: 0.738364, acc.: 40.00%] [G loss: 0.797612]
900 [D loss: 0.741788, acc.: 45.00%] [G loss: 0.816182]
901 [D loss: 0.745503, acc.: 42.50%] [G loss: 0.790426]
902 [D loss: 0.715379, acc.: 48.75%] [G loss: 0.